# Datos tickers

In [ ]:
import pandas as pd
import numpy as np
import yfinance as yf
from stockstats import StockDataFrame
from fredapi import Fred
import yesg
import requests
from tqdm.notebook import tqdm
from datetime import datetime, timedelta
from fake_useragent import UserAgent
import os
import time


def obtener_indicadores_tecnicos(ticker, start_date="2009-01-01", end_date=None, guardar_csv=False, ruta_csv=None):
    """
    Descarga precios e indicadores técnicos para un ticker,
    BUsca y guardando en CSV local y devuelve un DF con sufijos por columnas.
    """
    indicadores = [
        'macd', 'macds', 'macdh', 'rsi_14', 'adx', 'mfi',
        'boll', 'boll_ub', 'boll_lb', 'atr_14', 'close_14_roc', 'vr_20'
    ]

    start_dt = pd.to_datetime(start_date)
    end_dt = pd.to_datetime(end_date or datetime.today())
    origen = pd.to_datetime("2009-01-01")
    origen_ext  = origen - timedelta(days=30)
    final = pd.to_datetime(datetime.today())

    # Descarga o recarga CSV
    if (guardar_csv == True and ruta_csv == None):
        os.makedirs("data/precios", exist_ok=True)
    raw_csv = f"data/precios/{ticker}_prices.csv"

    if os.path.exists(raw_csv):
        df_stock = pd.read_csv(raw_csv)
    else:
        df = yf.download(ticker, start=origen_ext, end=final, auto_adjust=False, progress=False)
        if df.empty:
            print(f"No hay datos para {ticker}")
            return None

        if isinstance(df.columns, pd.MultiIndex):
            df.columns = df.columns.get_level_values(0)

        df.columns = [c.lower() for c in df.columns]
        df.index.name = 'date'

        # Cálculo de indicadores
        stock = StockDataFrame.retype(df.copy())
        for ind in indicadores:
            _ = stock[ind]
        stock['boll_width'] = (stock['boll_ub'] - stock['boll_lb']) / stock['boll']

        # Convertir a pandas, filtrar fecha y columnas base
        df_stock = pd.DataFrame(stock).reset_index()
        df_stock = df_stock[df_stock['date'] >= origen].reset_index(drop=True)

        cols = [
            'date', 'adj close', 'rsi_14', 'macd', 'macds', 'macdh',
            'atr_14', 'close_14_roc', 'adx', 'boll_width', 'mfi', 'vr_20', 'volume'
        ]
        df_stock = df_stock[cols]
        df_stock.rename(columns={'adj close': 'adj_close'}, inplace=True)

        # Añadir sufijo
        sufijos = {c: (f"{c}_{ticker}" if c == 'adj_close' else f"tecn_{c}_{ticker}") for c in df_stock.columns if c != 'date'}
        df_stock.rename(columns=sufijos, inplace=True)

        if guardar_csv:
            df_stock.to_csv(raw_csv, index=False)

    df_stock['date'] = pd.to_datetime(df_stock['date'])
    df_stock = df_stock[(df_stock['date'] >= start_dt) & (df_stock['date'] <= end_dt)].reset_index(drop=True)
    return df_stock

def obtener_dividendos(ticker, start_date="2009-01-01", end_date=None, guardar_csv=False, ruta_csv=None):
    """
    Descarga dividendos para un ticker, con periodicidad diaria,
    Busca y guardando en CSV local y devuelve un DF con sufijos por columnas.
    """
    start_dt   = pd.to_datetime(start_date)
    end_dt     = pd.to_datetime(end_date) if end_date else datetime.today().strftime('%Y-%m-%d')
    origen = pd.to_datetime("2009-01-01")
    final = pd.to_datetime(datetime.today())

    # Descarga o recarga CSV
    if (guardar_csv == True and ruta_csv == None):
        os.makedirs("data/dividendos", exist_ok=True)
    div_csv = f"data/dividendos/{ticker}_divids.csv"

    if os.path.exists(div_csv):
        df_div = pd.read_csv(div_csv)
    else:
        df = yf.Ticker(ticker).dividends.reset_index()
        if df.empty:
            print(f"No hay datos para {ticker}")
            return None

        df = df[['Date', 'Dividends']].copy()
        df.columns = ['date', f'dividendo_{ticker}']
        df['date'] = df['date'].dt.tz_localize(None)

        fechas_diarias = pd.DataFrame({'date': pd.date_range(start=origen, end=final, freq='D')})

        df_div = fechas_diarias.copy()
        df_div = df_div.merge(df, on='date', how='left')
        df_div.fillna(0.0, inplace=True)

        if guardar_csv:
            df_div.to_csv(div_csv, index=False)

    df_div['date'] = pd.to_datetime(df_div['date'])
    df_div = df_div[(df_div['date'] >= start_dt) & (df_div['date'] <= end_dt)].reset_index(drop=True)
    return df_div



def get_historic_esg_fixed(ticker):
    """
    Reparación de la funcion de yesg para conseguir datos historicos, ya que cambio el access point
    """
    url = "https://query2.finance.yahoo.com/v1/finance/esgChart"
    params = {"symbol": ticker}
    headers = {
        "User-Agent": "Mozilla/5.0"
    }

    response = requests.get(url, params=params, headers=headers)
    data = response.json()

    try:
        symbol_series = data["esgChart"]["result"][0]["symbolSeries"]
        timestamps = symbol_series["timestamp"]
        esg = symbol_series.get("esgScore", [])
        env = symbol_series.get("environmentScore", [])
        soc = symbol_series.get("socialScore", [])
        gov = symbol_series.get("governanceScore", [])

        df = pd.DataFrame({
            "Date": pd.to_datetime(timestamps, unit="s"),
            "Total-Score": esg,
            "E-Score": env,
            "S-Score": soc,
            "G-Score": gov
        }).set_index("Date")

        return df

    except Exception as e:
        print("Error al procesar los datos ESG:", e)
        return None

def obtener_esg(ticker, start_date="2009-01-01", end_date=None, guardar_csv=False, ruta_csv=None):
    """
    Retorna una serie temporal diaria de puntuaciones ESG para un ticker ,
    interpolando valores faltantes y combinando datos de yesg y Sustainalytics.

    Parámetros:
       ticker (str): símbolo del activo (por ejemplo, 'AAPL')
       start_date (str): fecha de inicio (formato 'YYYY-MM-DD')
       end_date (str or None): fecha de fin; por defecto, el día actual
    """

    start_dt = pd.to_datetime(start_date)
    end_dt = pd.to_datetime(end_date or datetime.today())
    fecha_salto = pd.to_datetime("2019-12-01")
    origen = pd.to_datetime("2009-01-01")
    final = pd.to_datetime(datetime.today())

    if (guardar_csv == True and ruta_csv == None):
        os.makedirs("data/esg", exist_ok=True)

    os.makedirs("data/originales", exist_ok=True)
    esg_csv = f"data/esg/{ticker}_esg.csv"
    hf_csv = f"data/originales/df_hf.csv"
    parquet_path = "hf://datasets/nlp-esg-scoring/spx-sustainalytics-esg-scores/data/train-00000-of-00001-7f89b36539207c5e.parquet"

    # Descarga o recarga CSV
    if os.path.exists(esg_csv):
        df_esg = pd.read_csv(esg_csv)
    else:

        # Datos Sustainalytics historico (hf)
        if os.path.exists(hf_csv):
            df_hf = pd.read_csv(hf_csv, parse_dates=['Date'])
        else:
            df_hf = pd.read_parquet(parquet_path)
            df_hf.to_csv(hf_csv, index=False)
            df_hf = pd.read_csv(hf_csv, parse_dates=['Date'])

        df_hf.columns = df_hf.columns.str.strip()
        df_hf.rename(columns={'Date': 'date', 'total_esg_score': 'esg_score',
                              'environment_score': 'ambiental','social_score': 'social',
                              'governance_score': 'gobernanza'}, inplace=True)
        df_hf['date'] = pd.to_datetime(df_hf['date'])

        # Datos de yesg
        fechas = pd.date_range(start=origen, end=final, freq='D')
        df_base = pd.DataFrame({'date': fechas})

        df_yesg = get_historic_esg_fixed(ticker)

        if df_yesg.empty:
            print(f"No hay datos de ESG para {ticker}")
            return None

        df_yesg.rename(columns={'Date':'date', 'Total-Score':'esg_score',
                                'E-Score':'ambiental', 'S-Score':'social',
                                'G-Score':'gobernanza'}, inplace=True)
        df_yesg = df_yesg.reset_index()
        df_yesg.rename(columns={df_yesg.columns[0]: 'date'}, inplace=True)
        df_yesg['date'] = pd.to_datetime(df_yesg['date'])

        # Combinar, Interpolar y remonbrado
        df_hft = df_hf[df_hf['Ticker'] == ticker].copy()
        df_comb = df_hft[['date', 'esg_score', 'ambiental', 'social', 'gobernanza']].copy()
        df_comb = df_comb.set_index('date').combine_first(df_yesg.set_index('date')).reset_index()

        antes = df_comb[df_comb['date'] <= fecha_salto - pd.Timedelta(days=1)]
        despues = df_comb[df_comb['date'] >= fecha_salto]

        df_antes = df_base[df_base['date'] <= fecha_salto - pd.Timedelta(days=1)].merge(antes, on='date', how='left')
        df_despues = df_base[df_base['date'] >= fecha_salto].merge(despues, on='date', how='left')

        for df in [df_antes, df_despues]:
            num_cols = df.select_dtypes(include=[np.number]).columns
            df[num_cols] = df[num_cols].interpolate(method='linear', limit_direction='both')

        columnas_esg = ['esg_score']
        df_despues[columnas_esg] = 100 - df_despues[columnas_esg] # IMPORTANTE: En la fecha de salto hay un cambio de CRITERIO.

        df_esg = pd.concat([df_antes[['date']+columnas_esg], df_despues[['date']+ columnas_esg]])

        for col in columnas_esg:
            if col in df_esg:
                df_esg.rename(columns={col: f"{col}_{ticker}"}, inplace=True)
        if guardar_csv:
            df_esg.to_csv(esg_csv, index=False)

    df_esg['date'] = pd.to_datetime(df_esg['date'])
    df_esg = df_esg[(df_esg['date'] >= start_dt) & (df_esg['date'] <= end_dt)].reset_index(drop=True)
    return df_esg

def obtener_fundamentales(ticker, guardar_csv=False, ruta_csv=None):
    """
    Obtiene información financiera básica de una empresa desde yfinance,
    renombrando las columnas con el sufijo _<ticker>.

    Parámetro:
        ticker (str): símbolo de cotización (ej. 'AAPL')
    """
    # Descarga o recarga CSV
    if (guardar_csv == True and ruta_csv == None):
        os.makedirs("data/fundamentales", exist_ok=True)
    fundam_csv = f"data/fundamentales/{ticker}_fundamentals.csv"

    if os.path.exists(fundam_csv):
        df_fundamentales = pd.read_csv(fundam_csv)
    else:
        try:
            info = yf.Ticker(ticker).info
            datos = {
                f"fund_marketCap_{ticker}": info.get("marketCap"),
                #f"sector_{ticker}": info.get("sector"),
                f"fund_beta_{ticker}": info.get("beta"),
                f"fund_ROE_{ticker}": info.get("returnOnEquity"),
            }
        except Exception as e:
            print(f"Error al obtener datos de {ticker}: {e}")
            return None

        df_fundamentales = pd.DataFrame([datos])
        if guardar_csv:
            df_fundamentales.to_csv(fundam_csv, index=False)

    return df_fundamentales


In [ ]:

def datos_ticker(ticker, start_date="2009-01-01", end_date=None, guardar_csv=False, ruta_csv=None):
    """
    Genera un dataset diario completo para un ticker que incluye:
    - Indicadores técnicos y precios
    - Dividendos diarios
    - Puntuaciones ESG interpoladas
    - Datos fundamentales replicados a cada fila diaria

    Parámetros:
        ticker (str): símbolo del activo (ej. 'AAPL')
        start_date (str): inicio del período (YYYY-MM-DD)
        end_date (str or None): fin del período, por defecto hoy
        guardar_csv (bool): si True, guarda el CSV final en disco
        ruta_csv (str or None): ruta opcional para guardar el archivo
    """

    start_dt = pd.to_datetime(start_date)
    end_dt = pd.to_datetime(end_date or datetime.today())
    origen = pd.to_datetime("2009-01-01")
    final = pd.to_datetime(datetime.today())

    if (guardar_csv == True and ruta_csv == None):
        os.makedirs("data/completos", exist_ok=True)

    total_csv = ruta_csv or f"data/completos/{ticker}_completo.csv"

    if os.path.exists(total_csv):
        print(f"Usando caché: {ticker}.csv")
        df_ticker = pd.read_csv(total_csv)
    else:
        print(f"Generando {ticker}...")

        df_tecnicos = obtener_indicadores_tecnicos(ticker, origen, final, guardar_csv = guardar_csv)
        df_dividendos = obtener_dividendos(ticker, origen, final, guardar_csv = guardar_csv)
        df_esg = obtener_esg(ticker, origen, final, guardar_csv = guardar_csv)
        df_fundamentales = obtener_fundamentales(ticker, guardar_csv = guardar_csv)

        for df in [df_tecnicos, df_dividendos, df_esg]:
            df['date'] = pd.to_datetime(df['date'])

        # Expandir datos fundamentales
        if df_fundamentales is not None:
            fechas = pd.DataFrame({'date': df_tecnicos['date']})
            df_fundamentales = fechas.merge(df_fundamentales, how='cross')

        # Unir todos los conjuntos
        df_ticker = df_tecnicos.merge(df_dividendos, on='date', how='left')\
                        .merge(df_esg, on='date', how='left')\
                        .merge(df_fundamentales, on='date', how='left')
        if guardar_csv:
            df_ticker.to_csv(total_csv, index=False)


    df_ticker['date'] = pd.to_datetime(df_ticker['date'])
    df_ticker = df_ticker[(df_ticker['date'] >= start_dt) & (df_ticker['date'] <= end_dt)].reset_index(drop=True)

    return df_ticker


def datos_grupo(tickers, start_date="2009-01-01", end_date=None, guardar_csv=False, ruta_csv=None):
    """
    Genera un DataFrame combinado en formato wide con datos diarios completos para múltiples tickers.
    Solo se incluyen fechas donde hay datos comunes para todos los tickers.

    Parámetros:
        tickers (list of str): lista de símbolos (ej. ['AAPL', 'MSFT'])
        start_date (str): fecha de inicio (YYYY-MM-DD)
        end_date (str or None): fecha de fin (por defecto hoy)
    """
    dfs = []

    if (guardar_csv == True and ruta_csv == None):
        os.makedirs("data/grupo", exist_ok=True)

    grupo_csv = ruta_csv or f"data/grupo/grupo_completo.csv"

    for ticker in tickers:
        df = datos_ticker(ticker, start_date=start_date, end_date=end_date, guardar_csv = guardar_csv)
        if df is None or df.empty:
            print(f"No se pudo obtener datos para {ticker}. Se omite.")
            continue
        dfs.append(df)

    if not dfs:
        print("No se generó ningún DataFrame.")
        return pd.DataFrame()

    df_merged = dfs[0]
    for df in dfs[1:]:
        df_merged = df_merged.merge(df, on='date', how='inner')

    if guardar_csv:
        df_merged.to_csv(grupo_csv, index=False)

    return df_merged


# Datos macroeconomicos

In [ ]:
def obtener_datos_macro(start_date="2009-01-01", end_date=None, guardar_csv=False, ruta_csv=None):

    your_key = "TU_API_KEY_AQUI" #ES NECESARIO DISPONER DE UN ACLAVE PROPIA DE ACCCESO A FRED PARA LA EXTRACCION DE DATOS

    start_dt = pd.to_datetime(start_date)
    end_dt = pd.to_datetime(end_date or datetime.today())
    origen = pd.to_datetime("2009-01-01")
    final = pd.to_datetime(datetime.today())

    end_date = end_date or datetime.today().strftime('%Y-%m-%d')

    if (ruta_csv == None):
        os.makedirs("data/macros", exist_ok=True)

    macro_csv = ruta_csv or f"data/macros/datos_macro.csv"

    if os.path.exists(macro_csv):
        print(f"Usando caché: datos_macro.csv")
        df_macro = pd.read_csv(macro_csv)
    else:
        print(f"Generando datos macroeconómicos")

        if not your_key:
            raise ValueError("Clave FRED API no disponible.")
        fred = Fred(api_key=your_key)

        def obtener_fred_diario(serie_id, nombre_columna):
            serie = fred.get_series(serie_id)
            df = pd.DataFrame(serie, columns=[nombre_columna])
            df.index = pd.to_datetime(df.index)
            df = df[origen:final].resample('D').interpolate(method='linear')
            return df

        fred_series = {
            'CPIAUCSL': 'macro_CPI', 'FEDFUNDS': 'macro_FED_RATE',
            'UNRATE': 'macro_UNEMPLOYMENT', 'INDPRO': 'macro_INDUSTRIAL_PRODUCTION', 'GDP': 'macro_GDP'
        }

        fred_dataframes = [obtener_fred_diario(sid, nombre) for sid, nombre in fred_series.items()]
        df_fred = pd.concat(fred_dataframes, axis=1)

        tickers_yahoo = {
            "^VIX": "macro_VIX", "CL=F": "macro_WTI_CRUDE", "GC=F": "macro_GOLD",
            "^GSPC": "macro_SP500", "^TNX": "macro_BOND_10Y",
            "EURUSD=X": "macro_EURUSD", "JPYUSD=X": "macro_JPYUSD", "GBPUSD=X": "macro_GBPUSD"
        }

        series_yahoo = []
        for ticker, nombre in tickers_yahoo.items():
            try:
                df = yf.download(ticker, start=origen, end=final, progress=False)
                if not df.empty and "Close" in df.columns:
                    serie = df["Close"].copy()
                    serie.name = nombre
                    series_yahoo.append(serie)
            except Exception as e:
                print(f"Error con {nombre}: {e}")

        df_yahoo = pd.concat(series_yahoo, axis=1)
        df_yahoo = df_yahoo.rename(columns=tickers_yahoo)
        df_macro = pd.concat([df_fred, df_yahoo], axis=1).interpolate('linear').bfill()

        df_macro.index.name = 'date'
        df_macro = df_macro.reset_index()
        if guardar_csv:
            df_macro.to_csv(macro_csv, index=False)

    df_macro['date'] = pd.to_datetime(df_macro['date'])
    df_macro = df_macro[(df_macro['date'] >= start_dt) & (df_macro['date'] <= end_dt)].reset_index(drop=True)
    return df_macro


def obtener_datos_fear_greed(start_date="2009-01-01", end_date=None, guardar_csv=False, ruta_csv=None):
    actual_date = '2020-08-01'
    start_dt = pd.to_datetime(start_date)
    end_dt = pd.to_datetime(end_date or datetime.today())
    origen = pd.to_datetime("2009-01-01")
    final = pd.to_datetime(datetime.today())

    if (ruta_csv == None):
        os.makedirs("data/macros", exist_ok=True)

    fear_csv = ruta_csv or f"data/macros/datos_fear.csv"
    archivo_antiguo = f"data/macros/all_fng_csv.csv"


    if os.path.exists(fear_csv):
        print(f"Usando caché: datos_fear.csv")
        df_fear = pd.read_csv(fear_csv)
    else:

        # Datos antiguos de archivo
        df_fng_antiguo = pd.read_csv(archivo_antiguo, usecols=["Date", "Fear Greed"]).rename(columns={"Date": "date", "Fear Greed": "macro_Fear_Greed"})
        df_fng_antiguo["date"] = pd.to_datetime(df_fng_antiguo["date"])
        df_fng_antiguo.set_index("date", inplace=True)
        df_fng_antiguo = df_fng_antiguo.replace(0, pd.NA)

        # Datos actuales
        BASE_URL = "https://production.dataviz.cnn.io/index/fearandgreed/graphdata/"
        ua = UserAgent()
        headers = {'User-Agent': ua.random}
        response = requests.get(BASE_URL + actual_date, headers=headers)
        data = response.json()
        if 'fear_and_greed_historical' not in data:
            raise ValueError("Error descargando Fear & Greed.")
        fear_greed = pd.DataFrame({datetime.fromtimestamp(entry['x'] / 1000).strftime('%Y-%m-%d'): int(entry['y']) for entry in data['fear_and_greed_historical']['data']}, index=["macro_Fear_Greed"]).T
        fear_greed.index = pd.to_datetime(fear_greed.index)

        df_fng_nuevo = fear_greed.reindex(pd.date_range(start=actual_date, end=final, freq='D'))
        full_index = pd.date_range(start=origen, end=final, freq='D')

        # Fusion
        df_fear = df_fng_antiguo.reindex(full_index)
        df_fear.update(df_fng_nuevo)
        df_fear["macro_Fear_Greed"] = pd.to_numeric(df_fear["macro_Fear_Greed"], errors="coerce")

        # Limpieza
        primer_valor = df_fear["macro_Fear_Greed"].first_valid_index()
        df_fear.loc[:primer_valor, "macro_Fear_Greed"] = df_fear.loc[:primer_valor, "macro_Fear_Greed"].fillna(50)
        df_fear["macro_Fear_Greed"] = df_fear["macro_Fear_Greed"].interpolate(method="linear")
        df_fear.index.name = "date"
        df_fear = df_fear.reset_index()

        if guardar_csv:
            df_fear.to_csv(fear_csv, index=False)

    df_fear['date'] = pd.to_datetime(df_fear['date'])
    df_fear = df_fear[(df_fear['date'] >= start_dt) & (df_fear['date'] <= end_dt)].reset_index(drop=True)
    return df_fear


# Integracion

In [ ]:
def obtener_dataset_completo( tickers, start_date="2009-01-01", end_date=None, guardar_csv=False, ruta_csv=None):

    if (ruta_csv == None):
        os.makedirs("data/total", exist_ok=True)

    ruta_csv = ruta_csv or f"data/total/datos_total.csv"

    df_tickers = datos_grupo(tickers, start_date, end_date, guardar_csv=False, ruta_csv=None )
    df_macro = obtener_datos_macro (start_date, end_date, guardar_csv=False, ruta_csv=None)
    df_fng = obtener_datos_fear_greed(start_date, end_date, guardar_csv=False, ruta_csv=None)

    # Asegurar tipos datetime
    df_tickers['date'] = pd.to_datetime(df_tickers['date'])
    df_macro['date'] = pd.to_datetime(df_macro['date'])
    df_fng['date'] = pd.to_datetime(df_fng['date'])

    # Unir todos los conjuntos
    df = df_tickers.merge(df_macro, on='date', how='inner')\
                    .merge(df_fng, on='date', how='inner')

    if guardar_csv:
        df.to_csv(ruta_csv, index=False)
        print(f"Dataset guardado en: {ruta_csv}")

    return df

In [ ]:
grupo1 = ["MSFT", "NVDA", "ADBE", "PGR",
          "AAPL", "PG", "MA", "AMGN",
          "XOM", "OXY", "LMT", "WFC"]

grupo2 = ["ORCL", "IBM", "CRM", "CMI",
          "GOOGL", "PEP", "UPS", "MDT",
          "CVX", "DVN", "BAC", "STLD"]

grupo3 = ["CSCO", "ACN", "HD", "REGN",
          "TXN", "NEE", "LVS", "FCX",
          "COP", "APA", "DAL", "GD"]

tickers = grupo1 + grupo2 + grupo3


grupo_mini = [
    "MSFT",  # Bueno – Tecnología
    "PGR",   # Bueno – Seguros
    "GOOGL", # Moderado – Comunicación / Tecnología
    "MDT",   # Moderado – Salud
    "XOM",   # Malo – Energía
    "WFC"    # Malo – Automoción / Industrial
]

df = obtener_dataset_completo(grupo_mini, start_date="2009-01-01", end_date="2025-05-14", guardar_csv=True, ruta_csv="data/total/datos_mini.csv")
print(df)

Generando MSFT...
Generando PGR...
Generando GOOGL...
Generando MDT...
Generando XOM...
Generando WFC...
Generando datos macroeconómicos
Dataset guardado en: data/total/datos_mini.csv
           date  adj_close_MSFT  tecn_rsi_14_MSFT  tecn_macd_MSFT  \
0    2009-01-02       14.924775         61.251191       -0.025845   
1    2009-01-05       15.064257         63.050074        0.045930   
2    2009-01-06       15.240449         65.244917        0.117633   
3    2009-01-07       14.322797         48.939459        0.085127   
4    2009-01-08       14.770607         54.867132        0.101656   
...         ...             ...               ...             ...   
4113 2025-05-09      438.730011         71.935811       14.841791   
4114 2025-05-12      449.260010         75.239253       16.117984   
4115 2025-05-13      449.140015         75.130722       16.924597   
4116 2025-05-14      452.940002         76.296774       17.666820   
4117 2025-05-15      452.420013         75.773213       1

In [ ]:
import pandas as pd
import numpy as np

# Comprovacion de nulos o inf

null_counts = df.isnull().sum()
inf_counts = df.isin([np.inf, -np.inf]).sum()

stats = pd.DataFrame({
    "column":   df.columns,
    "null_count": null_counts.values,
    "inf_count":  inf_counts.values
})

print(stats.to_string(index=False))


                     column  null_count  inf_count
                       date           0          0
             adj_close_MSFT           0          0
           tecn_rsi_14_MSFT           0          0
             tecn_macd_MSFT           0          0
            tecn_macds_MSFT           0          0
            tecn_macdh_MSFT           0          0
           tecn_atr_14_MSFT           0          0
     tecn_close_14_roc_MSFT           0          0
              tecn_adx_MSFT           0          0
       tecn_boll_width_MSFT           0          0
              tecn_mfi_MSFT           0          0
            tecn_vr_20_MSFT           0          0
           tecn_volume_MSFT           0          0
             dividendo_MSFT           0          0
             esg_score_MSFT           0          0
        fund_marketCap_MSFT           0          0
             fund_beta_MSFT           0          0
              fund_ROE_MSFT           0          0
              adj_close_PGR    